# Decision Trees Book Excerpt
*The following is from a book I am currently writing that will teach students the ins and outs of decision tree algorithms and demonstrate the techniques necessary to implement them performantly.*

*The material here covers Walter Fisher's suboptimzation lemma, which is the cornerstone of classic [CART](https://www.amazon.com/Classification-Regression-Wadsworth-Statistics-Probability/dp/0412048418/) categorical feature splits.*

*© Copyright James Dellinger, 2020-2025. All rights reserved.*

#### *Fisher, 1958* 
One final meta-question could be asked: how did we know we could employ the above shortcut for binary classification tasks? Why does sorting a categorical variable's levels in increasing probability of one of the classes drastically reduce the maximum number of splits we need to explore in order to find the best split? 

Well, back in 1958, Walter D. Fisher of Kansas State College [discovered](http://www.csiss.org/SPACE/workshops/2004/SAC/files/fisher.pdf) an optimization that has remained relevant for over half a century. Fisher was trying to devise an algorithm that would be able to partition a given amount of numerical samples into a certain quantity of subsets, all while ensuring that the samples were grouped together such that each subset's variance would be as small as possible.

Specifically, if there were $n$ samples, each with a value $a_{i}$, that were to be grouped into $k$ subsets, Fisher wanted to be able to group the samples together such that the sum of all samples' squared residuals, $\sum_{i=1}^{n}\left(a_{i}-\bar{a}_{subset}\right)$ was minimized. $\bar{a}_{subset}$ is the arithmetic mean of all samples of the subset in which sample $a_{i}$ was placed. Fisher calls these "least squares" partitions. 

#### Fisher's Suboptimization Lemma
Prior to this, if a practitioner had wanted to partition a set of $n$ numerical data points into $k$ groups and also ensure that they found the optimal grouping that minimized the sum of least squares of all samples, they would have had to comb through ${n-1\choose k-1}$ unique partitioning schemes. Fisher's breakthrough was the discovery a property that *drastically* reduces the amount of groupings that need to be searched through on the way toward locating the optimal partitions. 

Fisher describes this property in the "suboptimization lemma" that appears page seven of his [paper](http://www.csiss.org/SPACE/workshops/2004/SAC/files/fisher.pdf):
> Because of the additive character of the squared distance function, however, it is possible to reduce the computations very substantially by the use of suboptimization procedures. Such procedures are implied by the following lemma.
>
> *Suboptimization Lemma*: If $A_{1}\vdots A_{2}$ denotes a partition of set $A$ into two disjoint subsets $A_{1}$ and $A_{2}$, if $P^{*}_{1}$ denotes a least squares partition of $A_{1}$ into $G_{1}$ subsets and if $P^{*}_{2}$ denotes a least squares partition of $A_{2}$ into $G_{2}$ subsets; then, of the class of subpartitions $A_{1}\vdots A_{2}$ employing $G_{1}$ subsets over $A_{1}$ and $G_{2}$ subsets over $A_{2}$ a least squares partition is $P^{*}_{1}\vdots P^{*}_{2}$.

I found Fisher's language and notation to be opaque. And this is unfortunate because the property he discovered is really quite simple. Namely, when we use least squares as the measure of the "sameness" of the samples contained within a partition, the optimal partitions of an entire set of numerical samples will always be contiguous. This means that it's possible for us to use the results of a search that located $k-1$ optimal parititions when we try to find $k$ optimal partitions of the same set of $n$ samples. Obviously, knowing the output of searching for $k-2$ optimal partitions will help us discover the best partitioning for $k-1$ subsets, and results of finding $k-3$ partitions will help us find $k-2$ partitions, and so on and so forth. This pattern holds true all the way back to a base case that calculates the sum of least squares distances of placing a progressively increasing number of samples (from one sample all the way to $n$ samples) into just one group.

Note that while there may exist multiple optimal partitioning schemes, using Fisher's algorithm guarantees that we'll always be able to find one of them.

#### Dynamic Programming
Folks familiar with the technique will notice that the procedure I outlined just above sounds a lot like a description of a dynamic programming algorithm. Indeed, although Fisher doesn't mention the phrase "dynamic programming" by name anywhere in his paper, from the way he describes his algorithm on page 8, we can infer that this was precisely the approach he used.

Dynamic programming can be a little tough to get the hang of if you're not used to it. The idea is to first define and store the solution to a base case. Then you write an algorithm that calculates the state one step after the base case and stores that solution. You keep executing the algorithm to solve all subsequent steps, using solutions to previous steps as inputs and recording the solution to the latest step as you go along. The algorithm stops once the solution to the desired step has been reached. 

Before I demonstrate an implementation of Fisher's partitioning algorithm, let's look at two much more simple problems where dynamic programming can be utilized: calculating a factorial and computing a fibonacci number.

In [186]:
def factorial(n):
    """Dynamic programming algorithm for calculating the factorial of a given integer <n>.
    From example found at: https://www.geeksforgeeks.org/tabulation-vs-memoization/
    """
    dp = [0]*(n+1)
    dp[0] = 1 # Base case
    for i in range(1,n+1):
        dp[i] = dp[i-1] * i
    return dp[n]

factorial(6)

720

In [187]:
def fibonacci(n):
    """Dynamic programming algorithm for producing the fibonacci number of an integer <n>.
    From example found at: 
        https://www.geeksforgeeks.org/overlapping-subproblems-property-in-dynamic-programming-dp-1/
    """
    dp = [0]*(n+1)
    dp[0], dp[1], = 0, 1 # Base case
    for i in range(2, n+1):
        dp[i] = dp[i-1] + dp[i-2]
    return dp[n]

fibonacci(7)

13

#### Fisher Optimal Partition Dynamic Programming Algorithm
Below is my implementation of Fisher's algorithm to find $k$ optimal least squares partitions of $n$ items. It's an ideal application for showcasing the power of dynamic programming -- while a recursive solution would have an exponential computational complexity, using dynamic programming gives us an algorithm that finds the solution in only $O(kn^{2})$ time. 

The three helper functions immediately below are used for calculating the sum of squared residuals for a group of consecutive items.

In [188]:
# Look-up sum of elements between indices l and (including) r
def get_sum   (sums, l, r):          return sums[r+1] - sums[l]
    
# Look-up sum of squares of elements between indices l and (including) r
def get_sumsqr(sumsqrs, l, r):       return sumsqrs[r+1] - sumsqrs[l]

# Look-up sum of squared residuals for elements between indices l and (including) r
def get_SSR   (sums, sumsqrs, l, r): return get_sumsqr(sumsqrs, l, r) - get_sum(sums, l, r)**2/(r-l+1)

In [189]:
def FOP(items, k):
    """Implementation of Fisher's dynamic programming algorithm to 
    calculate the minimum cost and cut-points of 2 through k contiguous 
    least-squares partitions of a 1-d sequence of numbers.
    
    Arguments: 
        items (list): The numbers to be partitioned.
        k      (int): Number of desired partitions.
        
    Returns: 
       list containing the index of each partition's final item,
       sum of squared residuals of the optimal split
    """
    n = len(items)
    sums, sumsqrs = [0], [0] # Build lists of cumulative sums and cumulative sums of squares 
    for i in range(1, n+1):  # for computing sum of squared residuals between any two indices.
        sums.append(sums[-1] + items[i-1]); sumsqrs.append(sumsqrs[-1] + items[i-1]**2)

    # Each entry D[m][i] contains the cost (in our case, the minimum sum 
    # of least square distances) of grouping i+1 samples into k+1 partitions.
    # The bottom-right corner D[k-1][n-1] will contain the minimum cost of 
    # placing all n samples into contiguous least squares partitions.
    D = np.full((k,n), np.inf)

    # Each entry S[m][i] is the final cut point of the optimal grouping of i
    # samples into m partitions. 
    S = np.zeros((k,n), int)

    for i in range(n): D[0][i] = get_SSR(sums, sumsqrs, 0, i) # Base case: costs of 0 to i samples in left partition.
    for m in range(1,k):
        for i in range(m,n):
            for j in range(i, m-1, -1):
                cost = D[m-1][j-1] + get_SSR(sums, sumsqrs, j, i)
                if cost < D[m][i]: D[m][i], S[m][i] = cost, j-1

    cut_points, j = [], S[-1][-1] # Backtrack through the cut-point matrix S to get all k-1
    for m in range(k-1, 0, -1):   # cut-points for k optimal least squares partitions.
        cut_points.append(j); j = S[m-1][j]
        
    return [c for c in reversed(cut_points)], D[k-1][n-1] # Return cut points and total cost (SSR) of split

There are many sources that attempt to explain Fisher's optimal partitioning algorithm using mathematical notation and or pseudocode. Unfortunately, these either contain small mistakes that will nonetheless lead the uninitiated astray, or, they simultaneously use overly complex math notation while at the same time include nothing more than vague pseudocode that glosses over crucial details.

The exception to this was a [paper](https://journal.r-project.org/archive/2016/RJ-2016-022/RJ-2016-022.pdf) by Tibor Szkaliczki that adequately explained, step-by-step, the nuts and bolts of the optimal partitioning dynamic programming algorithm. This resource was invaluable in helping me finally unlock enough of my own understanding so as to be able to write the Python implementation just above.

#### Atlanta Income Example
To verify that my implementation works as expected, I'll reproduce solutions to the two examples Fisher uses in his paper. The first of these is a task of grouping Atlanta families together based on the similarity of their income classes. There are a total of ten income classes, and each income class is defined in terms of average dollar income per family.

<p style="align: middle; font-size: 9pt; text-align: center; color: #686868; margin-bottom: 0.5em;"><img src="images/atlanta_income.png" alt="Atlanta Family Income Graph" align="middle" width="60%">Ten 1933 Average Income Cohorts of Atlanta Families. Printed on page 3 of Fisher's 1958 <a href= "http://www.csiss.org/SPACE/workshops/2004/SAC/files/fisher.pdf"><font color=#686868>paper</font></a>.</p>

In [190]:
incomes = [236, 335, 454, 606, 869, 1248, 1753, 2372, 3558, 6766]

Fisher refers to the Atlanta income task as an "unrestricted" problem. This is because there is nothing keeping us from organizing the list of ten income classes into any order we want before searching for an optimal partition. If we want to be sure that our partitioning algorithm locates the optimal least squares partitions of the income levels, we must first sort these in order of least to greatest.

In [191]:
cut_points, cost = FOP(incomes, 3)
print(cut_points, cost)

[5, 8] 2395281.3333333335


Note that each cut point is the index of the final sample located in a single partition. Here are the income classes contained in each of the three optimal partitions:

In [192]:
print(incomes[0:cut_points[0]+1], incomes[cut_points[0]+1:cut_points[1]+1], incomes[cut_points[1]+1:])

[236, 335, 454, 606, 869, 1248] [1753, 2372, 3558] [6766]


These results are identical to the 3-partition solution that Fisher reports on page 3 of the paper: the 6 smallest classes go into one group, the next three go into the second group, and the largest income class sits by itself inside the third partition.

#### Lake Michigan-Huron Lake Levels
For the Lake Michigan-Huron problem, the task is to group the lake's level measurements from years 1860 through 1955 into separate partitions where the measurements inside each partition are as similar as possible. Each year's measurement is the maximum of the 12 monthly averages for a single year. Unlike the task for Atlanta income classes, these lake levels *can't* be sorted in order from least to greatest. They must remain orderd by year, and it's due to the presence of this requirement that Fisher refers to this kind of task as a "restricted" problem. 

<div style="display:table;">
  <div style="display:table-cell; width: 60%; vertical-align:bottom;">
      <p style="font-size: 9pt; text-align: center; width: 100%; color: #686868; margin-bottom: 0.5em;"><img src="images/lake_michigan_huron.png" alt="Lake Michigan-Huron Graph">Lake Michigan-Huron Yearly Average of Monthly Lake Level Highs. <br>Printed on page 5 of Fisher's 1958 <a href= "http://www.csiss.org/SPACE/workshops/2004/SAC/files/fisher.pdf"><font color=#686868>paper</font></a>.</p>
  </div>
  <div style="display:table-cell;vertical-align:bottom">
     <p style="font-size: 9pt; text-align: center; width: 100%; color: #686868; margin-bottom: 0.5em;"><img src="images/lake_mich_huron_data.jpg" alt="Lake Michigan-Huron Data">Lake Michigan-Huron Level Data. Printed on page 566 of <br><a href= "https://www.amazon.com/Statistics-Approach-W-Allen-Wallis/dp/0029337208"><font color=#686868>Statistics: A New Approach</font></a> by Wallis and Roberts, 1956.</p>
  </div> 
</div>

Even though the lake levels must remain sorted by year, and we thus won't be able to find the truly optimal least squares partitioning of all these numbers, the good news is that Fisher's algorithm will still help us find an optimal parititioning that exists within the confines imposed by the task's sorting restriction.

In [193]:
lake_levels = [583.3, 583.5, 583.2 , 582.6 , 582.2, 582.1 , 581.7 , 582.2 , 581.6 , 582.1, # 1860's
               582.7, 582.8, 581.5 , 582.2 , 582.3, 582.1 , 583.6 , 582.7 , 582.5 , 581.5, # 1870's
               582.1, 582.2, 582.6 , 583.3 , 583.1, 583.3 , 583.7 , 582.9 , 582.3 , 581.8, # 1880's
               581.6, 580.9, 581.0 , 581.3 , 581.4, 580.2 , 580.0 , 580.85, 580.83, 581.1, # 1890's
               580.7, 581.1, 580.83, 580.82, 581.5, 581.6 , 581.5 , 581.6 , 581.8 , 581.1, # 1900's
               580.5, 580.0, 580.7 , 581.3 , 580.7, 580.0 , 581.1 , 581.87, 581.91, 581.3, # 1910's
               581.0, 580.5, 580.6 , 579.8 , 579.6, 578.49, 578.49, 579.6 , 580.6 , 582.3, # 1920's
               581.2, 579.1, 578.6 , 578.7 , 578.0, 578.6 , 578.7 , 578.6 , 579.7 , 580.0, # 1930's
               579.3, 579.0, 580.2 , 581.5 , 580.8, 581.0 , 580.96, 581.1 , 580.8 , 579.7, # 1940's
               580.0, 581.6, 582.7 , 582.1 , 581.7, 581.5]                                 # 1950's

In [194]:
cut_points, cost = FOP(lake_levels, 10)
print(cut_points, cost)

[2, 22, 28, 62, 67, 70, 77, 81, 90] 23.592421857523732


Happily, the cost and cut points for a 10-way partition of the yearly lake level data are once again identical to that which Fisher reports on page 6 of his paper.

#### From Fisher's Lemma to Breiman's Proof
Now the key takeaway from the income and lake level examples we walked through above is this: because we use least squares as the measure of similarity, or pureness, of the samples inside a single partition, we enjoy a significant reduction in the number splits that must be explored if we want to find the optimal partitioning system that would make the samples inside each partition as similar to each other as possible. Instead of having to sample the full [Stirling number](https://en.wikipedia.org/wiki/Stirling_numbers_of_the_second_kind) of unique partitioning schemes, we can get by with sampling *far* fewer splits. For example, when searching for the optimal 2-way partition of a list of $L$ elements, the number of splits that must be explored reduces from $2^{L-1}-1$ to $L-1$.

As Philip A. Chou underscores on page 2 of his 1991 [paper](https://pdfs.semanticscholar.org/9576/c0f4202ea4e6b30f9c13b397e3ccc5476250.pdf), this is because a least squares partitioning of a list of numbers always leads to contiguous partitions. If we're facing an unrestricted problem and we can first sort the elements in least-to-greatest order, we're guaranteed to find an optimal partitioning. 

Chou goes on to explain that what Breiman accomplished in the proof on pages 274 to 278 of the CART book was the confirmation that even if a concave measure of partition purity like the gini score is being used, and even if we're working with samples that are just ones and zeroes (i.e. a binary classification task), that the implications of Fisher's suboptimization lemma still apply. All we have to do is first replace each one of a feature's $L$ categories with the expected value of its label (will be somewhere between 0 and 1, and is determined by the labels of the samples that have that particular value for that feature). We then sort these $L$ expected values from least to greatest, and then search over their $L-1$ splits to find the 2-way split with the lowest gini score.

#### Andy Liaw
I'd like to briefly pause and provide some background on the practitioner today who, aside from Adele Cutler, may have had the closest correspondence with Leo Breiman while Breiman was refining the random forest algorithm during what would be the final few years of his life. In 2002, [Andy Liaw](https://www.linkedin.com/in/andy-liaw-1399347/) was a research scientist at Merck and he, along with fellow Merck scientist Matthew Wiener, created an [R interface](https://www.rdocumentation.org/packages/randomForest/versions/4.6-14) for Cutler and Breiman's original Fortran 77 implementation of random forests.

In their subsequent [paper](http://cogns.northwestern.edu/cbmg/LiawAndWiener2002.pdf) that covered how to use their R `randomForest` package for classification, regression, and unsupervised learning, Liaw and Wiener indicated that not only had Leo Breiman made available to them random forests' Fortran 77 source code, but that he had also taken the time several of their questions regarding finer points of its implementation details. Over the intervening years, Andy Liaw rewrote the original Fortran 77 code [in C](https://github.com/cran/randomForest/tree/master/src), and today he is [listed](https://www.rdocumentation.org/packages/randomForest/versions/4.6-14) as the package's sole maintainer. 

What's more, although there are currently more performant R implementations of random forests that are growing in popularity, such as [Marvin Wright's](https://mnwright.github.io/) [Ranger library](https://github.com/imbs-hl/ranger) that's optimized for high-dimensional data sets, Liaw's tried-and-true `randomForest` package still may possibly be the most widely-used random forests package in the R ecosystem. 

As for how this all relates to me, I don't particularly enjoy reading Fortran code so Liaw's [C version](https://github.com/cran/randomForest/tree/master/src) served as a priceless reference as I sought to understand *precisely* how the mechanics of Breiman and Cutler's categorical split finding algorithms worked.

Finally, here is a talk Liaw gave at the [R Conference](https://rstats.ai/) in April, 2018 in New York City. After introducing random forests, he touches on some of the unique advantages that make the algorithm stand out even amongst modern popular approaches. Chief among these pros is the fact that random forests simply doesn't overfit.

In [195]:
IFrame("https://www.youtube.com/embed/XJnjlpW9w5A?rel=0&amp;controls=1&amp;showinfo=0", width = 560, height = 315)

#### The "Absent Levels" Problem
Unfortunately, beyond the inconveniences of potentially longer than ideal computation times and an unavoidable cap on categorical variable cardinality, Breiman's classic CART approach to categorical splits has a more serious achilles heel.

In 2018, Google researcher Timothy C. Au published an in-depth [study](https://dl.acm.org/doi/abs/10.5555/3291125.3309607) of a phenomonenon he termed the "absent levels" problem. In a nutshell, this is the tendency of a tree-based model's prediction to become biased if an observation is passed down a node with a categorical split that doesn't explicitly designate how to handle the value belonging to that observation. In other words, this is what can happen when an input from the test set has a value for a categorical feature that wasn't seen among the training samples originally used to split the node when the tree was grown.

Recall that when we implemented multi-class CART categorical splits, in order to find the best split at a given node we performed an exhaustive search over the possible 2-way splits of *all* the levels of a categorical feature, regardless of whether or not some of those levels were absent from the training samples in the mother node. On the one hand, a benefit of this kind of exhaustive search is that the node's best categorical split will be able to handle at inference inputs possessing levels originally absent from the mother node during training.

The obvious downside, however, is that the chosen split will *always* send any and all missing levels to the right side. In contrast, the same split will always send at least one of the present levels to the left side. This should intuitively strike us as a bit fishy -- we now have a situation where categorical splits handle levels differently based on the single criterion of whether those levels were present or absent during training. Despite the claim that all splitting decisions are based solely on gini impurity, there is clearly another source of information (whether the level was present or absent during training) that is unintentionally influencing split choices. Absent levels are sent right not because doing this decreased gini impurity. Instead, they're handled this way due to the simple fact that they were absent from the mother node during training. 

What's more, the categorical split shortcut for binary classification tasks is also not immune to the absent levels problem. Au explains that in this case absent levels are sent to the left child by default. In addition to this tendency, the fact that (owing to the way the shortcut works) levels on the left side of these splits are less likely to be classified as having a label of `0` (where class labels are `0` and `1`) does nothing but add fuel to the bias fire. Absent levels are systematically more likely to be classified as a `1` than a `0`, simply because they were absent from the parent node.

To sum things up, the root of the bias that Au exposes in his paper is this:

>Present levels: splitting decision is based on decrease of gini impurity.<br>
Absent levels: splitting decision is based *only* on the fact that the level was absent.

Au found that in practice this bias could result in model accuracy that's either serendipitously good or tragically sub-par, and that it all depends on the nature of the dataset and task at hand. In some scenarios, it will turn out to have been a good thing that absent levels are handled in the naive way that Breiman, Cutler, and Liaw's algorithm specifies. Other times, however, predictions would have been more accurate had the algorithm naively chosen to send absent levels to the other side of categorical splits that's opposite of the naive default.

To illustrate this, Au ran a series of experiments on real datasets using Liaw's `randomForest` R package. Au compared the outcome the package's default naive handling of absent levels ("Left" on binary classification or "Right" for multi-class tasks) with that of handling them in the equally naive but opposite manner. He then constrasted both of these results with those of various other possible strategies for getting around the absent levels bias.

These heuristics include things like one-hot encoding all categorical features before training ("One-Hot"), as well as techniques used to generate predictions when observations have a missing value for feature used to split a decision tree node. Three such strategies are (1) setting an observation's prediction to the label held by most of training samples originally in the mother node ("Stop"), sending the observation into whichever child node got the most samples during training ("Majority"), or sending the observation into one of the two child nodes at random with a weighting determined by the proportion of the parent node's training samples received by each child node ("Random"). 

<p style="align: middle; font-size: 9pt; text-align: center; color: #686868; margin-bottom: 0.5em;"><img src="images/promesa_au.png" alt="PROMESA results" align="middle" width="60%">Results for the PROMESA binary classification task. Printed on page 25 of Au's 2018 <a href= "https://dl.acm.org/doi/abs/10.5555/3291125.3309607"><font color=#686868>paper</font></a>.</p>

In experiments using 2-level (political party) and 50-level (home state) categorical predictors, along with two ordered variables that attempt to quantify a politician's ideology, to predict whether a U.S. House Rep's vote on the Puerto Rico Oversight, Management, and Economic Stability Act (PROMESA) was "No" or "Yes", Au found that the default approach of sending absent levels to the left child grossly underperformed the reverse approach of sending them all to the right.

The takeaway *isn't* that it's better to always do the polar opposite of whatever `randomForest` does by default. Rather, the key point is that for any given data set, without adequate experimentation and comparison there's no way to know if it'll be more beneficial to naively send all absent levels to the left side of a split, or to the right. Based on the results of the above experiment as well as two others, Au ultimately recommends making weighted random choices about whether to send an observation containing an absent level to the left or right side of a categorical split. He shows that the "Random" technique gives results that are similar to or better than most of the other heuristics. Most importantly, the "Random" approach doesn't run the risk of delivering catastrophically bad results that can occur when the wrong choice is made about which way to naively send observations with absent levels.

In 2019, Marvin Wright and Inke König [recommended](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6368971/pdf/peerj-07-6339.pdf) another promising approach to categorical splits that avoids the absent levels problem. Before I explain their recommendations, I'd like to quickly touch on two other widely-used (but less than ideal) techniques for using categorical predictors with decision trees.